# Import Packages

In [ ]:
from __future__ import absolute_import, division, print_function, unicode_literals
# basic python package
import importlib
import numpy as np
import time
import logging
importlib.reload(logging)
logging.basicConfig(level = logging.INFO)

# python ploting packages
import matplotlib.pyplot as plt
from matplotlib.ticker import MaxNLocator
from matplotlib.colors import ListedColormap, LinearSegmentedColormap, BoundaryNorm
from matplotlib.collections import LineCollection
from matplotlib import cm


# self-define classes
from script import CR_ML_Class as CR
from script import load_mock_data as LD


# tensorflow
import tensorflow as tf
from tensorflow.keras.models import Sequential, load_model
print("Tensorflow Version is {}".format(tf.__version__))
print("Keras Version is {}".format(tf.keras.__version__))